<a href="https://colab.research.google.com/github/JYPIT/DataScience/blob/main/Practice/%EB%84%A4%EC%9D%B4%EB%B2%84_%EC%98%81%ED%99%94_TOP100_%ED%81%AC%EB%A1%A4%EB%A7%81%26%EC%98%81%ED%99%94_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#링크 수집을 위한 라이브러리 설치
# install chromium, its driver, and selenium

!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
# set options to be headless, ..
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver',options=options)


Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 http://ppa.launchpad.net/graph

In [ ]:
from bs4 import BeautifulSoup
import requests 
from selenium import webdriver as wd
from selenium.webdriver.common.keys import Keys
import time
import re
import pandas as pd     

In [ ]:
# https://serieson.naver.com/movie/top100List.nhn?rankingTypeCode=PC_R 실시간
# https://serieson.naver.com/movie/top100List.nhn?rankingTypeCode=PC_D 일간
# https://serieson.naver.com/movie/top100List.nhn?rankingTypeCode=PC_W 주간
# https://serieson.naver.com/movie/top100List.nhn?rankingTypeCode=PC_M 월간

def getPageLinksWantRange(startPageNo, lastPageNo):
  links = [] # 100개의 영화 링크를 수집할 리스트 변수 선언
  #return_links = []

  for pageNo in range(startPageNo-1, lastPageNo):
    # 실시간 TOP 100 영화 링크 수집
    url = "https://serieson.naver.com/movie/top100List.nhn?&rankingTypeCode=PC_W&page=" + str(pageNo + 1)

    req = requests.get(url) # 페이지 접속
    soup = BeautifulSoup(req.text, 'lxml') # html 파싱을 위한 객체 생성
    movielinks = soup.select('div.lst_thum_wrap ul li a[href]') # <div class="lst_thum_wrap"> -> <ul> -> <li> -> <a href ="">

    for movielink in movielinks:
      link = str(movielink.get('href'))
      links.append("https://series.naver.com" + link) # 앞에 https:// ~~를 붙여주기 위함이다.

  return links

###네이버 영화 제목, 평점, 장르, 줄거리 크롤링


In [ ]:
def getMovieDataFromNaverSeries(links):
  title_infos = [] # 제목
  content_infos = [] # 줄거리
  genre_infos = [] # 장르
  score_infos = [] # 평점
  date_infos = [] # 개봉일

  url2 = "https://www.naver.com" # 정상적인 접속인 척

  driver = webdriver.Chrome('chromedriver',options=options)
  driver.get(url2)
  time.sleep(3.0) #3초 sleep (시간을 벌어준다.)

  driver.find_element_by_css_selector('body').send_keys(Keys.CONTROL + "t") # 새탭을 연다. (사람인척)

  for link in links:
    print(link + ' 수집 중.......')
    driver.switch_to.window(driver.window_handles[-1]) # 새창 활성화
    time.sleep(0.1)
    driver.get(link)
    time.sleep(0.1)
    driver.switch_to.window(driver.window_handles[0]) # 새창 활성화
    time.sleep(0.3)

    html_source = driver.page_source # req.text 같은 동작

    html_soup = BeautifulSoup(html_source,'lxml')
    
    #청소년 관람불가 영화 크롤링 할 경우 -> 인증 -> 수집 제외
    
    flag = html_soup.text[0:10]
    
    newflag = "".join(flag)
    newflag = newflag.replace('\n','')

    if newflag == '네이버':
      time.sleep(1.0)

      #평점 수집
      score = driver.find_element_by_css_selector('div.score_area > em')
      score = float(score.text)
      score = int(score)
      score_infos.append(score)
      print(score)

      #장르 수집

      genre = driver.find_element_by_css_selector('li.info_lst > ul > li:nth-child(4)').get_attribute('textContent')
      genre = genre.replace('장르','')
      genre = genre.replace('\n','')
      genre = genre.replace('\t','')
      genre_infos.append(genre)
      print(genre)

      # 제목 줄거리
      try:
        movieInfoUrl = driver.find_element_by_css_selector('li.info_lst > ul > li:nth-child(7) > a').get_attribute('href')
        
      except:
        movieInfoUrl = driver.find_element_by_css_selector('li.info_lst > ul > li:nth-child(6) > a').get_attribute('href')                         

      movie_req = requests.get(movieInfoUrl)

      # 제목 수집
      movie_soup = BeautifulSoup(movie_req.text, 'lxml')
      title = movie_soup.head.find("meta", {"property":"og:title"}).get('content')
      title_infos.append(title)
      print(title)

      # 줄거리 수집
      contents_texts = movie_soup.select('div.story_area > p.con_tx')

      if len(contents_texts) == 0:
        content_infos.append("줄거리 오류")
      
      else:
        for contents in contents_texts:
          temp = contents.text
          temp = temp.replace('\r','') # 줄바꿈 제거
          temp = temp.replace('\xa0','') # 공백을 나타내는 것을 제거
          content_infos.append(temp)

    elif newflag == '' :
      print("청불 영화로 데이터 수집하지 않습니다.")
    
  print("수집 완료 합니다.....")
  print(len(score_infos), len(genre_infos), len(content_infos))
  
  driver.close()
  
  movie_dic = {"제목":title_infos, "평점":score_infos,"장르":genre_infos,"줄거리":content_infos}
  movie_df = pd.DataFrame(movie_dic)

  #중복데이터 삭제 (구매/대여)
  movie_df2 = movie_df.drop_duplicates("줄거리",keep='first') # 줄거리 비교해서 영화를 삭제(첫번째 수집 영화만 남김)

  return movie_df2

In [ ]:
#csv 파일로 저장
def dftoCsv(movie_df,num):
  try:
    movie_df.to_csv(('movie_my_data'+ str(num) +'.csv'),sep=',',na_rep='NaN',encoding='euc=kr')
  except:
    print("Error")

In [ ]:
# 1) 크롤링 할 링크 수집
links = getPageLinksWantRange(1,5)

# 2) 영화 상세 정보 수집
movie_df2 = getMovieDataFromNaverSeries(links)

# 3) 파일 저장
dftoCsv(movie_df2, len(movie_df2))


https://series.naver.com/movie/detail.nhn?productNo=6291892 수집 중.......
9
애니메이션
극장판 귀멸의 칼날: 무한열차편
https://series.naver.com/movie/detail.nhn?productNo=3400212 수집 중.......
7
공포/스릴러
콰이어트 플레이스
https://series.naver.com/movie/detail.nhn?productNo=2822635 수집 중.......
청불 영화로 데이터 수집하지 않습니다.
https://series.naver.com/movie/detail.nhn?productNo=3024534 수집 중.......
8
액션
킬러의 보디가드
https://series.naver.com/movie/detail.nhn?productNo=6283368 수집 중.......
6
액션
파이프라인
https://series.naver.com/movie/detail.nhn?productNo=6023769 수집 중.......
9
애니메이션
소울
https://series.naver.com/movie/detail.nhn?productNo=3400211 수집 중.......
7
공포/스릴러
콰이어트 플레이스
https://series.naver.com/movie/detail.nhn?productNo=3639460 수집 중.......
청불 영화로 데이터 수집하지 않습니다.
https://series.naver.com/movie/detail.nhn?productNo=6243813 수집 중.......
9
액션
크루즈 패밀리: 뉴 에이지
https://series.naver.com/movie/detail.nhn?productNo=6119777 수집 중.......
7
드라마
미나리
https://series.naver.com/movie/detail.nhn?productNo=6139375 수집 중.......
9
애니메이션
소울
https://series.naver.co

###크롤링한 TOP 100 영화 데이터 랭킹 시각화

In [22]:
import pandas as pd
df1 = pd.read_csv('/content/movie_my_data_75_R.csv',encoding='euc-kr')
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75 entries, 0 to 74
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  75 non-null     int64 
 1   제목          75 non-null     object
 2   평점          75 non-null     int64 
 3   장르          75 non-null     object
 4   줄거리         75 non-null     object
dtypes: int64(2), object(3)
memory usage: 3.1+ KB


In [32]:
import plotly.express as px
df1 = pd.read_csv('/content/movie_my_data_75_R.csv',encoding='euc-kr')
print(df1.head(5))
print(df1.tail(5))
# /content/movie_my_data_75_R.csv
# /content/movie_my_data_63_W.csv
# /content/movie_my_data_64_M.csv
# /content/movie_my_data_75_R.csv

   Unnamed: 0  ...                                                줄거리
0           0  ...  소리를 내는 순간 공격받는 극한의 상황 속에서 살아남기 위한 한 가족의 숨막히는 사...
1           1  ...  혈귀로 변해버린 여동생 ‘네즈코’를 인간으로 되돌릴 단서를 찾아 비밀조직 귀살대에 ...
2           2  ...  세상 제일 잘난 맛에 사는 섭외 1순위 앵그리 보디가드가지명수배 1순위 구강 액션 ...
3           3  ...  처음부터 난 알았어. 내가 특별하단 걸그게 불편한 인간들도 있겠지만 모두의 비위를 ...
4           4  ...  손만 대면 대박을 터트리는 도유 업계 최고 천공기술자 ‘핀돌이’는수천억의 기름을 빼...

[5 rows x 5 columns]
    Unnamed: 0  ...                                                줄거리
70          79  ...  저 마코토에게는 남들에게 말 못할 비밀이 하나 있어요. 타임리프라고 하는 능력이죠....
71          80  ...  말괄량이 바이올라(아만다 바인즈)는 헤어진 남자친구에게 복수하기 위하여 쌍둥이 오빠...
72          81  ...  해리 포터 일생일대 최대 난관!요즘 들어 매일 꾸는 악몽 때문에 이마의 상처에 더욱...
73          82  ...  조엘은 아픈 기억만을 지워준다는 라쿠나사를 찾아가 헤어진 연인 클레멘타인의 기억을 ...
74          83  ...  200년간 지속된 십자군 전쟁이 한창이었던 12세기 예루살렘.운명에 이끌려 십자군 ...

[5 rows x 5 columns]


In [33]:
fig = px.scatter(df1,x="제목",y="평점",color='장르')
fig.show()

In [34]:
fig = px.bar(df1,x="제목",y="평점",color = '장르')
fig.show()

In [35]:
fig = px.pie(df1, values='평점',names='장르',title="실시간 Top 100")
fig.show()

###DAILY

In [38]:
import pandas as pd
import plotly.express as px

df2 = pd.read_csv('/content/movie_my_data_61_D.csv',encoding='euc-kr')
print(df2.info())

print(df2.head(5))
print(df2.tail(5))
# /content/movie_my_data_75_R.csv
# /content/movie_my_data_61_D.csv
# /content/movie_my_data_63_W.csv
# /content/movie_my_data_64_M.csv

fig = px.scatter(df2,x="제목",y="평점",color='장르')
fig.show()

fig = px.bar(df2,x="제목",y="평점",color = '장르')
fig.show()

fig = px.pie(df2, values='평점',names='장르',title="일간 Top 100")
fig.show()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  61 non-null     int64 
 1   제목          61 non-null     object
 2   평점          61 non-null     int64 
 3   장르          61 non-null     object
 4   줄거리         61 non-null     object
dtypes: int64(2), object(3)
memory usage: 2.5+ KB
None
   Unnamed: 0  ...                                                줄거리
0           0  ...  혈귀로 변해버린 여동생 ‘네즈코’를 인간으로 되돌릴 단서를 찾아 비밀조직 귀살대에 ...
1           1  ...  소리를 내는 순간 공격받는 극한의 상황 속에서 살아남기 위한 한 가족의 숨막히는 사...
2           2  ...  세상 제일 잘난 맛에 사는 섭외 1순위 앵그리 보디가드가지명수배 1순위 구강 액션 ...
3           3  ...  처음부터 난 알았어. 내가 특별하단 걸그게 불편한 인간들도 있겠지만 모두의 비위를 ...
4           4  ...  뉴욕에서 음악 선생님으로 일하던 ‘조’는꿈에 그리던 최고의 밴드와 재즈 클럽에서 연...

[5 rows x 5 columns]
    Unnamed: 0  ...                                                줄거리
56          70  ...  싱어송라이터인 ‘그레타’(키이라 나이틀리)는 남자친구

###WEEKLY

In [39]:
import pandas as pd
import plotly.express as px

df2 = pd.read_csv('/content/movie_my_data_63_W.csv',encoding='euc-kr')
print(df2.info())

print(df2.head(5))
print(df2.tail(5))
# /content/movie_my_data_75_R.csv
# /content/movie_my_data_63_W.csv
# /content/movie_my_data_64_M.csv
# /content/movie_my_data_75_R.csv

fig = px.scatter(df2,x="제목",y="평점",color='장르')
fig.show()

fig = px.bar(df2,x="제목",y="평점",color = '장르')
fig.show()

fig = px.pie(df2, values='평점',names='장르',title="주간 Top 100")
fig.show()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63 entries, 0 to 62
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  63 non-null     int64 
 1   제목          63 non-null     object
 2   평점          63 non-null     int64 
 3   장르          63 non-null     object
 4   줄거리         63 non-null     object
dtypes: int64(2), object(3)
memory usage: 2.6+ KB
None
   Unnamed: 0  ...                                                줄거리
0           0  ...  혈귀로 변해버린 여동생 ‘네즈코’를 인간으로 되돌릴 단서를 찾아 비밀조직 귀살대에 ...
1           1  ...  소리를 내는 순간 공격받는 극한의 상황 속에서 살아남기 위한 한 가족의 숨막히는 사...
2           2  ...  세상 제일 잘난 맛에 사는 섭외 1순위 앵그리 보디가드가지명수배 1순위 구강 액션 ...
3           3  ...  손만 대면 대박을 터트리는 도유 업계 최고 천공기술자 ‘핀돌이’는수천억의 기름을 빼...
4           4  ...  뉴욕에서 음악 선생님으로 일하던 ‘조’는꿈에 그리던 최고의 밴드와 재즈 클럽에서 연...

[5 rows x 5 columns]
    Unnamed: 0  ...                                                줄거리
58          73  ...  가족의 생계를 책임지기 위해 열심히 살아가는 ‘진태’

###MONTHLY


In [41]:
import pandas as pd
import plotly.express as px

df4 = pd.read_csv('/content/movie_my_data_64_M.csv',encoding='euc-kr')
print(df4.info())

print(df4.head(5))
print(df4.tail(5))
# /content/movie_my_data_75_R.csv
# /content/movie_my_data_61_D.csv
# /content/movie_my_data_63_W.csv
# /content/movie_my_data_64_M.csv

fig = px.scatter(df4,x="제목",y="평점",color='장르')
fig.show()

fig = px.bar(df4,x="제목",y="평점",color = '장르')
fig.show()

fig = px.pie(df4, values='평점',names='장르',title="월간 Top 100")
fig.show()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  64 non-null     int64 
 1   제목          64 non-null     object
 2   평점          64 non-null     int64 
 3   장르          64 non-null     object
 4   줄거리         64 non-null     object
dtypes: int64(2), object(3)
memory usage: 2.6+ KB
None
   Unnamed: 0  ...                                                줄거리
0           0  ...  "미나리는 어디서든 잘 자라"낯선 미국, 아칸소로 떠나온 한국 가족.가족들에게 뭔가...
1           1  ...  비범한 과거를 숨긴 채 남들과 다를 바 없는 평범한 일상을 사는 한 가정의 가장 ‘...
2           2  ...  “이 양반은 대역 죄인이니 너무 잘해줄 생각들 말어”순조 1년, 신유박해로 세상의 ...
3           3  ...  인간과 드래곤이 평화롭게 공존하던 신비의 땅, 쿠만드라 왕국살아있는 모든 생명을 삼...
4           4  ...  뉴욕에서 음악 선생님으로 일하던 ‘조’는꿈에 그리던 최고의 밴드와 재즈 클럽에서 연...

[5 rows x 5 columns]
    Unnamed: 0  ...                                                줄거리
59          71  ...  쇼 비즈니스의 창시자이자, 꿈의 무대로 전세계를 매료

In [ ]:
import pandas as pd
import plotly.express as px

df3 = pd.read_csv('/content/movie_my_data_64_.csv',encoding='euc-kr')
print(df3.info())

print(df3.head(5))
print(df3.tail(5))

fig = px.scatter(df3,x="제목",y="평점",color='장르')
fig.show()

fig = px.bar(df3,x="제목",y="평점",color = '장르')
fig.show()

fig = px.pie(df3, values='평점',names='장르',title="월간 Top 100")
fig.show()